In [1]:
# example from: https://machinelearningmastery.com/tutorial-to-implement-k-nearest-neighbors-in-python-from-scratch/
# github: https://github.com/madhug-nadig/Machine-Learning-Algorithms-from-Scratch/blob/master/K%20Means%20Clustering.py

In [2]:
import import_ipynb
from sklearn.datasets import load_iris
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score
import pandas as pd
import numpy as np
from math import sqrt
from numpy.linalg import inv
from scipy.interpolate import lagrange
import random
from network import NetworkNode, NetworkShare, merge, reconstruct
from smpc_secrets import ShamirSecretSharing, AdditiveSecretSharing, Vandermonde, P, RandPoly


In [6]:
# calculate the Euclidean distance between two vectors
# each row is the set of feataures for a node
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

In [7]:
def generate_functions(features_arr):
    all_functions = []
    for feature in range(len(features_arr)):
        func = RandPoly(name=f"f{feature}", n=1, 
                        R=[(i,x) for i, x in enumerate(list(
            [features_arr[feature],random.randint(2,250)]))]).poly
        all_functions.append(func)
    return all_functions

def generate_shares(func_array, node_id):
    shares = []
    for func in func_array:
        shares.append(func(node_id))
    return shares

def get_feature_distances(arr1, arr2):
    distances = []
    for feature_a, feature_b in zip(arr1, arr2):
        dist = (feature_a-feature_b)**2
        distances.append(dist)
    return distances

def sum_distances(arr):
    return sum(arr)

def reconstruct(shares):
    x = np.arange(1, len(shares) + 1)
    y = shares
    f = lagrange(x, y)
    return f(0)
    
def simplified_calc(alice_data, bob_data):
    # Alice's functions
    assert len(alice_data) == len(bob_data), "feature length mismatch!"

    # get private functions
    alice_functions = generate_functions(alice_data)
    bob_functions = generate_functions(bob_data)

    # get shares
    alice_personal_shares = generate_shares(alice_functions, 1)
    alice_from_bob = generate_shares(bob_functions, 1)

    bob_personal_shares = generate_shares(bob_functions, 2)
    bob_from_alice = generate_shares(alice_functions, 2)

    server_from_alice = generate_shares(alice_functions, 3)
    server_from_bob = generate_shares(bob_functions, 3)

    # compute distance for each feature
    alice_distances = get_feature_distances(alice_personal_shares, alice_from_bob)
    bob_distances = get_feature_distances(bob_personal_shares, bob_from_alice)
    server_distances = get_feature_distances(server_from_alice, server_from_bob)

    # get sum of distances
    alice_sum = sum_distances(alice_distances)
    bob_sum = sum_distances(bob_distances)
    server_sum = sum_distances(server_distances)

    print(alice_sum, bob_sum, server_sum)


    dist_squared = reconstruct([alice_sum, bob_sum, server_sum])

    alice_bob_distance = np.sqrt(dist_squared)

    return alice_bob_distance

In [12]:
a = [0,0]
b = [3,3]

print("regular calc: ", euclidean_distance(a,b), "\n")
simplified_calc(a,b)


regular calc:  4.242640687119285 

26545 103690 231453


4.242640687119285